In [1]:
import os
import re 
import logging as log
import requests

import json
from json.decoder import JSONDecodeError

from datetime import datetime, timedelta
import time
import glob

import numpy as np
import pandas as pd
from pandas import json_normalize

from bs4 import BeautifulSoup
from nltk.stem import WordNetLemmatizer


import yfinance as yf
from pandas_datareader import data as pdr

from datetime import datetime
from dateutil.relativedelta import *

from pygooglenews import GoogleNews
# from urllib3.exceptions import ProtocolError

pd.set_option('display.max_columns', 500)

In [192]:
def priceTracker(s):
    url = 'https://finance.yahoo.com/quote/{}?p={}&.tsrc=fin-srch'.format(s, s)

    # real-time price tracker

    while url:
        try:
            response = requests.get(url)

            soup = BeautifulSoup(response.text, 'lxml')
            
            tracker = soup.find_all('div', {'class': 'My(6px) Pos(r) smartphone_Mt(6px)'})
            
            tracker = tracker[0].find_all('span')
            current_price = tracker[0].text
            price_change = tracker[1].text

            price_change_dir = re.findall(r'\((.*?) *\)',price_change)[0][0]
            price_change_val = float(re.findall("\d+\.\d+", re.findall(r'\((.*?) *\)',price_change)[0])[0])
            print('{} current price: {}'.format(s, current_price))
            
            time.sleep(1)
            return current_price, price_change_dir, price_change_val
            break
        except:
            print("Connection refused by the server..")
            print("Let me sleep for 5 seconds")
            print("ZZzzzz...")
            
            time.sleep(5)
            print("Assign NaN values...")
            current_price = np.nan
            price_change_dir = np.nan
            price_change_val = np.nan
            print('{} current price: {}'.format(s, current_price))
            
            return current_price, price_change_dir, price_change_val
            break

In [150]:
priceTracker('ROKU')

ROKU current price: 178.27


('178.27', '-', 4.8)

### Get popular stocks with sector/industry/sub_industry info

In [6]:
stocks_info = pd.read_csv('screener_results_all.csv', header = 0)
stocks_info.shape

(5009, 9)

### Get stock lists from StockTwits

In [7]:
stocktwits_lists = pd.read_csv('Symbols_2020-09-24.csv', names=['Symbol', 'Name', 'Sector', 'Industry'])
print(stocktwits_lists.shape)


(14431, 4)


## Combine stocks list from StockTwits and Fidelity

In [8]:
stocks_combine_list = pd.merge(stocktwits_lists['Symbol'], stocks_info, 
                               how = 'inner', on = 'Symbol').reset_index(drop=True)
print(stocks_combine_list.shape)

stocks_combine_list.head()

(4684, 9)


,Symbol,Name,Security_Type,Security_Price,Sector,Industry,Sub_Industry,Exchange,Headquarters_Location
0,AAU,Almaden Minerals Ltd,Common Stock,0.425,Materials,Metals & Mining,Diversified Metals & Mining,AMEX,Canada
1,ACU,Acme United Corp,Common Stock,22.640,Industrials,Commercial Services & Supplies,Office Services & Supplies,AMEX,United States of America
2,ACY,AeroCentury Corp,Common Stock,1.250,Industrials,Trading Companies & Distributors,Trading Companies & Distributors,AMEX,United States of America
3,AE,Adams Resources & Energy Inc.,Common Stock,29.000,Energy,"Oil, Gas & Consumable Fuels",Oil & Gas Refining & Marketing,AMEX,United States of America
4,AGX,Argan Inc,Common Stock,34.620,Industrials,Construction & Engineering,Construction & Engineering,NYSE,United States of America


## Get 'Market Cap' from Yahoo Finance

In [139]:
def get_market_cap(s):

    url = 'https://finance.yahoo.com/quote/{}?p={}&.tsrc=fin-srch'.format(s, s)
        
    while url:
        try:
            response = requests.get(url)

            soup = BeautifulSoup(response.text, 'lxml')
            tracker = soup.find_all('td', {'class': 'Ta(end) Fw(600) Lh(14px)', 'data-reactid': ['138', '137']})
            market_cap = tracker[0].find_all('span', {'class': 'Trsdu(0.3s)'})[0].text
            break
        except:
            print("Connection refused by the server..")
            print("Let me sleep for 5 seconds")
            print("ZZzzzz...")
            time.sleep(5)
            print("Was a nice sleep, now let me continue the next one...")
            break

    return market_cap


## Add Market Cap in stocks list

In [153]:
'''
    reading mktcap
    
'''

f = open('mktcap', 'r')

lines = []
c = 0
for line in f:
    if c % 5 == 3:
        lines.append(line.strip().split('\t'))
    c += 1
print(len(lines))

df_mktcap = pd.DataFrame.from_records(lines, \
                            columns=['Symbol', 'Market_Cap', 'Price', 'p1', 'p2', 'p3'])
df_mktcap.drop(['Price', 'p1', 'p2', 'p3'], axis = 1, inplace = True)
df_mktcap['Market_Cap'] = df_mktcap['Market_Cap'].map(lambda x:\
                                    float(re.findall(r'\d+(?:\.\d+)?', x.replace(',', ''))[0]))

print(df_mktcap.shape)
df_mktcap.head()

3953
(3953, 2)


,Symbol,Market_Cap
0,AAPL,1.876243e+12
1,MSFT,1.537671e+12
2,AMZN,1.512583e+12
3,GOOGL,9.677772e+11
4,BABA,7.297916e+11


In [154]:
df_mktcap_combine = pd.merge(stocks_combine_list, df_mktcap, how = 'inner', on = 'Symbol')

print(stocks_combine_list.shape, df_mktcap.shape, df_mktcap_combine.shape)

df_mktcap_combine.head()

(4684, 9) (3953, 2) (3512, 10)


,Symbol,Name,Security_Type,Security_Price,Sector,Industry,Sub_Industry,Exchange,Headquarters_Location,Market_Cap
0,AAU,Almaden Minerals Ltd,Common Stock,0.425,Materials,Metals & Mining,Diversified Metals & Mining,AMEX,Canada,98108095.0
1,AE,Adams Resources & Energy Inc.,Common Stock,29.000,Energy,"Oil, Gas & Consumable Fuels",Oil & Gas Refining & Marketing,AMEX,United States of America,85906170.0
2,AGX,Argan Inc,Common Stock,34.620,Industrials,Construction & Engineering,Construction & Engineering,NYSE,United States of America,656259600.0
3,AIM,Aim Immunotech Inc,Common Stock,2.300,Health Care,Biotechnology,Biotechnology,AMEX,United States of America,80570700.0
4,AIM,Aim Immunotech Inc,Common Stock,2.300,Health Care,Biotechnology,Biotechnology,AMEX,United States of America,80570700.0


## Select a sample of stocks for the following analysis

1. Only keep mid-cap (2-10 billion) and large-cap (>= 10 billion) companies

2. Select randomly 10%/15% market-cap companies from each 'sub industry' or 'industry'

    micro-cap (< 300 million)
    
    small-cap (>=300 million, < 2 billion)
    
    mid-cap (2-10 billion)
    
    large-cap (>= 10 billion)
    

In [156]:
stocks_market_cap_dict = {
    'Symbol': [],
    'Size': [],
    'Market_Cap': []
}

a = df_mktcap_combine[['Symbol', 'Market_Cap']]

for i in range(a.shape[0]):
    m = a.loc[i, 'Market_Cap']
    s = a.loc[i, 'Symbol']
    val = m
        
    if val < 300*1000000:
        stocks_market_cap_dict['Symbol'].append(s)
        stocks_market_cap_dict['Size'].append('micro-cap')
        stocks_market_cap_dict['Market_Cap'].append(val)        
    if val >= 300*1000000 and val < 2*1000000000:
        stocks_market_cap_dict['Symbol'].append(s)
        stocks_market_cap_dict['Size'].append('small-cap')
        stocks_market_cap_dict['Market_Cap'].append(val)
    if val >= 2*1000000000 and val < 10*1000000000:
        stocks_market_cap_dict['Symbol'].append(s)
        stocks_market_cap_dict['Size'].append('mid-cap')
        stocks_market_cap_dict['Market_Cap'].append(val)
    if val >= 10*1000000000:
        stocks_market_cap_dict['Symbol'].append(s)
        stocks_market_cap_dict['Size'].append('large-cap')
        stocks_market_cap_dict['Market_Cap'].append(val)



In [157]:
stocks_market_cap_df = pd.DataFrame(stocks_market_cap_dict)

df_mid_large_stocks = stocks_market_cap_df[(stocks_market_cap_df['Size'] == 'mid-cap')|\
                    (stocks_market_cap_df['Size'] == 'large-cap')].reset_index(drop=True)

df_mid_large_stocks = pd.merge(df_mid_large_stocks, df_mktcap_combine, 
                               how = 'left', on = ['Symbol', 'Market_Cap'])

print(df_mid_large_stocks.shape)

df_mid_large_stocks.head()

(1470, 11)


,Symbol,Size,Market_Cap,Name,Security_Type,Security_Price,Sector,Industry,Sub_Industry,Exchange,Headquarters_Location
0,APO,large-cap,1.731101e+10,Apollo Global Management Inc,Common Stock,45.08,Financials,Capital Markets,Asset Management & Custody Banks,NYSE,United States of America
1,BKR,large-cap,1.376959e+10,Baker Hughes a GE Co,Common Stock,14.62,Energy,Energy Equipment & Services,Oil & Gas Equipment & Services,NYSE,United States of America
2,BLD,mid-cap,5.228389e+09,TopBuild Corp,Common Stock,93.66,Consumer Discretionary,Household Durables,Homebuilding,NYSE,United States of America
3,BTI,large-cap,8.224864e+10,British American Tobacco PLC,Depository Receipt,38.10,Consumer Staples,Tobacco,Tobacco,NYSE,United Kingdom
4,COR,mid-cap,4.966141e+09,CoreSite Realty Corp,Common Stock (REIT),124.13,Real Estate,Equity Real Estate Investment Trusts (REITs),Specialized REITs,NYSE,United States of America


In [161]:
stocks_market_cap_df['Size'].value_counts()

small-cap    1084
micro-cap     960
mid-cap       829
large-cap     639
Name: Size, dtype: int64

In [158]:
df_mid_large_stocks['Size'].value_counts()

mid-cap      829
large-cap    641
Name: Size, dtype: int64

In [160]:
'''
    stratify sampling:
        for number of stocks in Sub_Industry >= 10, 
        select 10%/15% stocks

'''

subIndustry_cnts = df_mid_large_stocks['Sub_Industry'].value_counts()

tmp_df = df_mid_large_stocks[df_mid_large_stocks['Sub_Industry'].isin(subIndustry_cnts[subIndustry_cnts >= 10].index)]

print(tmp_df.shape, df_mid_large_stocks.shape)

tmp_df['Size'].value_counts()

(1082, 11) (1470, 11)


mid-cap      597
large-cap    485
Name: Size, dtype: int64

In [163]:
stocks_midlargeCap_webscrape_df = tmp_df.groupby('Sub_Industry').apply(lambda x: \
        x.sample(frac = 0.15, random_state=123)).sort_values('Symbol').reset_index(drop=True)

print(stocks_midlargeCap_webscrape_df.shape)

stocks_midlargeCap_webscrape_df.head()

(167, 11)


,Symbol,Size,Market_Cap,Name,Security_Type,Security_Price,Sector,Industry,Sub_Industry,Exchange,Headquarters_Location
0,AAPL,large-cap,1.876243e+12,Apple Inc,Common Stock,315.01,Information Technology,"Technology Hardware, Storage & Peripherals","Technology Hardware, Storage & Peripherals",NASDAQ,United States of America
1,AB,mid-cap,2.589199e+09,AllianceBernstein Holding LP,Unit Trust Fund,22.51,Financials,Capital Markets,Asset Management & Custody Banks,NYSE,United States of America
2,ABT,large-cap,1.786819e+11,Abbott Laboratories,Common Stock,96.07,Health Care,Health Care Equipment & Supplies,Health Care Equipment,NYSE,United States of America
3,ACC,mid-cap,4.614801e+09,American Campus Communities Inc,Common Stock (REIT),30.59,Real Estate,Equity Real Estate Investment Trusts (REITs),Residential REITs,NYSE,United States of America
4,ADBE,large-cap,2.243249e+11,Adobe Inc,Common Stock,371.42,Information Technology,Software,Application Software,NASDAQ,United States of America


In [164]:
stocks_midlargeCap_webscrape_df.to_csv('df_stocks_midlargeCap_webscrape.csv')

## Construct a group of key words for each stocks

# Ticker + Company name!!!
narrow down key words, focus on company level

In [165]:
stocks_combine_list = pd.read_csv('stocks_midlargeCap_webscrape_df.csv', index_col=0)
print(stocks_combine_list.shape)

stocks_combine_list.head()

(167, 11)


,Symbol,Size,Market_Cap,Name,Security_Type,Security_Price,Sector,Industry,Sub_Industry,Exchange,Headquarters_Location
0,AAPL,large-cap,1.876243e+12,Apple Inc,Common Stock,315.01,Information Technology,"Technology Hardware, Storage & Peripherals","Technology Hardware, Storage & Peripherals",NASDAQ,United States of America
1,AB,mid-cap,2.589199e+09,AllianceBernstein Holding LP,Unit Trust Fund,22.51,Financials,Capital Markets,Asset Management & Custody Banks,NYSE,United States of America
2,ABT,large-cap,1.786819e+11,Abbott Laboratories,Common Stock,96.07,Health Care,Health Care Equipment & Supplies,Health Care Equipment,NYSE,United States of America
3,ACC,mid-cap,4.614801e+09,American Campus Communities Inc,Common Stock (REIT),30.59,Real Estate,Equity Real Estate Investment Trusts (REITs),Residential REITs,NYSE,United States of America
4,ADBE,large-cap,2.243249e+11,Adobe Inc,Common Stock,371.42,Information Technology,Software,Application Software,NASDAQ,United States of America


In [166]:
del_words = ['Ltd', 'Inc', 'Corp', 'Holdings', 'LLC', 'Corporation', 
             'HOLDERs', 'Co', 'Limited', 'Company',
             'Providers', 'group', 'com', 'Brands']
del_words = [w.lower() for w in del_words]

In [167]:
stocks_key_words_dict = {}

for s in stocks_combine_list['Symbol']:
    w_lemmatizer = WordNetLemmatizer()

    tmp_words = []
    tmp_words.append(s.lower())
    tmp_names = [w_lemmatizer.lemmatize(w.lower()) for w in re.findall(r'\w+', stocks_combine_list[stocks_combine_list['Symbol'] == s]['Name'].values[0]) \
                 if w.lower() not in del_words]
    tmp_words.extend(tmp_names)
    tmp_words.append(' '.join(tmp_names))
    
    tmp_words = list(set([w_lemmatizer.lemmatize(w) for w in tmp_words]))
    stocks_key_words_dict[s] = tmp_words
    

In [251]:
with open('/Users/lin/Downloads/Q3_Web_scraping/stocks_key_words_dict.json', 'w') as fp:
    json.dump(stocks_key_words_dict, fp, indent = 4)


## Get historical stocks data (past 6m) to decide the threshold for large change

In [ ]:
### compare with yesterday's close price???(open price)

In [170]:
## Get 12 months historical stock data, and cut at top_q% to determine a large change for the stock
def get_threshold_at_p(s, ms, top_p):
    # print(s)

    tmp_stock = yf.Ticker(s)

    current_date = datetime.today().strftime("%Y-%m-%d")
    start_date = (datetime.today() + relativedelta(months=-ms)).strftime("%Y-%m-%d")

    # print(current_date, start_date)

    tmp_hist = pdr.get_data_yahoo(s, start = start_date, end = current_date)
    tmp_hist['%change'] = round(abs(tmp_hist['Close'] - tmp_hist['Open'])/tmp_hist['Open']*100, 2)
    tmp_hist['change_dir'] = ['+' if tmp_hist['Close'][x] > tmp_hist['Open'][x] else '-' \
                              for x in range(tmp_hist.shape[0])]
    
    threshold_at_p = np.percentile(tmp_hist['%change'], top_p)
    # print(threshold_at_p)

    return threshold_at_p, tmp_hist

In [171]:
get_threshold_at_p('AAPL', ms = 12, top_p = 90)[0]

2.9480000000000004

## News API: extract news headlines

Use pygooglenews to extract Google News headlines


doc: https://github.com/kotartemiy/pygooglenews

The returned object contains "feed" (FeedParserDict) and "entries" list of articles found with all data parsed.

Accepted topics are:
1. WORLD
2. NATION
3. BUSINESS
4. TECHNOLOGY
5. ENTERTAINMENT
6. SCIENCE
7. SPORTS
8. HEALTH


gn.search(query: str, helper = True, when = None, from_ = None, to_ = None, proxies=None, scraping_bee=None)


"when" parameter (str) sets the time range for the published datetime.
- "h" for hours (worked for up to 101h). when = 12h will search for only the articles matching the search criteri and published for the last 12 hours
- "d" for days.
- "m" for month (worked for up to 48m)

"from_" and "to_" accept the following formate of date: "%Y-%m-%d" for example, "2020-07-01"

Tip. If you want to build a $\bf{\text{near real-time}}$ feed for a specific topic, use when='1h'. If Google captured fewer than 100 articles over the past hour, you should be able to retrieve all of them.

In [ ]:
## time_len -> 15/30mins less noise!!! -> 1 hour

In [172]:
def get_news(s, stocks_key_words_dict, time_len = '1h'):
    gn = GoogleNews()
    tmp_news = gn.search(' OR '.join(stocks_key_words_dict[s]), when = '1h')
    return tmp_news['entries']

In [176]:
## FB!
gn = GoogleNews()
tmp_news = gn.search(' OR '.join(['aapl', 'apple']), when = '1h')
len(tmp_news['entries']),\

(43,)

In [177]:
stocks_key_words_dict['AAPL']

['apple', 'aapl']

# Get UGC data from StockTwits

In [178]:
class Requests():
    """ Uses `requests` library to GET and POST to Stocktwits, and also to convert resonses to JSON
    """
    def get_json(url, params=None):
        """ Uses tries to GET a few times before giving up if a timeout.  returns JSON
        """
        resp = None
        for i in range(4):
            try:
                resp = requests.get(url, params=params, timeout=5)
            except requests.Timeout:
                trimmed_params = {k: v for k, v in params.items() if k not in ST_BASE_PARAMS.keys()}
                log.error('GET Timeout to {} w/ {}'.format(url[len(ST_BASE_URL):], trimmed_params))
            if resp is not None:
                break
        if resp is None:
            log.error('GET loop Timeout')
            return None
        else:
            return json.loads(resp.content)

    def post_json(url, params=None, deadline=30):
        """ Tries to post a couple times in a loop before giving up if a timeout.
        """
        resp = None
        for i in range(4):
            try:
                resp = requests.post(url, params=params, timeout=5)
            except requests.Timeout:
                trimmed_params = {k: v for k, v in params.iteritems() if k not in ST_BASE_PARAMS.keys()}
                log.error('POST Timeout to {} w/ {}'.format(url[len(ST_BASE_URL):], trimmed_params))
            if resp is not None:
                break
        # TODO wrap in appropriate try/except
        return json.loads(resp.content)


In [179]:
# StockTwits details
ST_BASE_URL = 'https://api.stocktwits.com/api/2/'
ST_BASE_PARAMS = dict(access_token=os.getenv('ST_ACCESS_TOKEN'))

TOTAL_LIMITS = 30

# ---------------------------------------------------------------------
# Basic StockTwits interface
# ---------------------------------------------------------------------

def get_stock_stream(symbol, params={}):
    """ gets stream of messages for given symbol
    """
    all_params = ST_BASE_PARAMS.copy()
    for k, v in params.items():
        all_params[k] = v
    return Requests.get_json(ST_BASE_URL + 'streams/symbol/{}.json'.format(symbol), params=all_params)


# Algorithm 

1. Get past 1 year historical stock data and decide the threshold based on top_p% percentile: 

get_threshold_at_p(s, ms = 12, top_p = 90): threshold_at_q

2. Real-time stock price tracker: return real-time stock price and % change between the current price and the open price: 

priceTracker(s) with time.sleep(): price_change_val


3. If price_change_val > threshold_at_q, start scaping the related news headline (containing stock name, sector, industry and sub_industry key words) and end up collecting the headline data after 30mins/1hour. And also get UGC data from StockTwits:

stocks_key_words_dict


In [181]:
stocks_combine_list['Sub_Industry'].value_counts()[:5]

Application Software     10
Biotechnology             9
Regional Banks            6
Health Care Equipment     5
Semiconductors            5
Name: Sub_Industry, dtype: int64

In [182]:
stocks_combine_list.head()

,Symbol,Size,Market_Cap,Name,Security_Type,Security_Price,Sector,Industry,Sub_Industry,Exchange,Headquarters_Location
0,AAPL,large-cap,1.876243e+12,Apple Inc,Common Stock,315.01,Information Technology,"Technology Hardware, Storage & Peripherals","Technology Hardware, Storage & Peripherals",NASDAQ,United States of America
1,AB,mid-cap,2.589199e+09,AllianceBernstein Holding LP,Unit Trust Fund,22.51,Financials,Capital Markets,Asset Management & Custody Banks,NYSE,United States of America
2,ABT,large-cap,1.786819e+11,Abbott Laboratories,Common Stock,96.07,Health Care,Health Care Equipment & Supplies,Health Care Equipment,NYSE,United States of America
3,ACC,mid-cap,4.614801e+09,American Campus Communities Inc,Common Stock (REIT),30.59,Real Estate,Equity Real Estate Investment Trusts (REITs),Residential REITs,NYSE,United States of America
4,ADBE,large-cap,2.243249e+11,Adobe Inc,Common Stock,371.42,Information Technology,Software,Application Software,NASDAQ,United States of America


# Make directory for today's results

- 3_Readme.txt: pulling stock price every 15/30mins

In [236]:
current_dt = datetime.today().strftime('%Y-%m-%d')
print(current_dt)

top_p = 90
print(top_p)

2020-09-28
90


In [237]:
# create file for each stocks
parent_dir = '/Users/lin/Downloads/Q3_Web_scraping/'
print(parent_dir)

tmp_path = os.path.join(parent_dir, 'mid_large_cap_stocks')
try:
    os.chdir(tmp_path)
except:
    os.mkdir(tmp_path)

tmp_path = os.path.join(tmp_path, current_dt)

try:
    os.chdir(tmp_path)
except:
    os.mkdir(tmp_path)
    
tmp_path = os.path.join(tmp_path, 'threshold_{}'.format(top_p))

try:
    os.chdir(tmp_path)
except:
    os.mkdir(tmp_path)
    os.chdir(tmp_path)

    
for s in stocks_combine_list['Symbol']:
    if not os.path.exists(os.path.join(tmp_path, s)):
        os.mkdir(os.path.join(tmp_path, s))
print(os.getcwd())


/Users/lin/Downloads/Q3_Web_scraping/
/Users/lin/Dropbox/My Mac (余振平的MacBook Pro)/Downloads/Q3_Web_scraping/mid_large_cap_stocks/2020-09-28/threshold_90


In [239]:
def return_large_change_stocks(thre_at_q_lists, current_price_change_lists, tmp_prices):
    
    tmp_stocks = []
    tmp_news_dict = {}
    tmp_ugc_dict = {}
    
    tmp_time = datetime.now().isoformat()
    
        
    for i, thre in enumerate(thre_at_q_lists):
        if current_price_change_lists[i] > thre:
            tmp_stocks.append((stocks_combine_list['Symbol'][i], tmp_prices[i][2], current_price_change_lists[i], tmp_time))
    
    if not tmp_stocks:
        print('No large change stocks at time {}'.format(tmp_time))
    else:
        print('Find large change stocks: {} at time {}'.format(tmp_stocks, tmp_time))
        
    return tmp_stocks

In [240]:
def save_webscape_data(cnt, tmp_stocks):
    tmp_time = datetime.now().isoformat()
        
    if not tmp_stocks:
        print('Iteration {}: No large change stocks found at time {}'.format(cnt, tmp_time)) 
    
    else:
        print('Iteration {}: Find large change stocks: {}'.format(cnt, tmp_stocks))
        for s in tmp_stocks:

            # News data
            tmp_news_dict = get_news(s, stocks_key_words_dict)
            with open(os.path.join(os.path.join(tmp_path, s), \
                               'News_{}_{}.json'.format(s, tmp_time)), 'w') as f:
                json.dump(tmp_news_dict, f, indent = 2)
            f.close()


            # StockTwits UGC data
            tmp_ugc_dict = get_stock_stream(s)
            with open(os.path.join(os.path.join(tmp_path, s), \
                               'UGC_{}_{}.json'.format(s, tmp_time)), 'w') as f:
                json.dump(tmp_ugc_dict, f, indent = 2)
            f.close()

In [241]:
print(datetime.now(), datetime.now() + timedelta(hours=5/60))

2020-09-28 20:58:19.806749 2020-09-28 21:03:19.806752


#### Get past 12months historical data

In [198]:
tmp_list = stocks_combine_list['Symbol'].values # sub_df['Symbol']

print(top_p)

thre_at_q_lists = [get_threshold_at_p(s, ms = 12, top_p=top_p)[0] for s in tmp_list]

df_thre_at_q = pd.DataFrame({'symbols': tmp_list, 
              'threshold_at_{}'.format(top_p): thre_at_q_lists})
df_thre_at_q[df_thre_at_q['threshold_at_90'] != df_thre_at_q['threshold_at_90']]

,symbols,threshold_at_90


In [242]:
df_thre_at_q.head()

,symbols,threshold_at_90
0,AAPL,2.948
1,AB,3.910
2,ABT,3.048
3,ACC,4.280
4,ADBE,3.300


In [226]:
# df_thre_at_q.to_csv('{}/df_thre_at_q_day{}.csv'.format(parent_dir, current_dt))

In [244]:
'''
    time_interval: 15mins, 30mins
'''
time_interval = 15

In [ ]:
# Comments

dt_end = datetime.now() + timedelta(hours=6.5)

print('starting time: {} and end time: {}'.format(datetime.now(), dt_end), '\n')

cnt = 1

current_prices = {}

large_change_stocks_all = {}

while datetime.now() < dt_end:
       
    start_time = time.time()

    print('iteration = {}'.format(cnt))

    tmp_stocks = []

    print(datetime.now())

    current_price_change_lists = []
    tmp_prices = []

    for s in tmp_list:
        tmp_pricetracker = priceTracker(s)
        current_price_change_lists.append(tmp_pricetracker[2])
        tmp_prices.append((s, datetime.now().isoformat(), tmp_pricetracker[0]))

    try:
        if not np.isnan(current_price_change_lists).any():
            tmp_stocks.extend(return_large_change_stocks(thre_at_q_lists, current_price_change_lists, tmp_prices))

        print('Checking all the stocks...')

        large_change_stocks_all[cnt] = tmp_stocks
        current_prices[cnt] = tmp_prices

        # Scraping UGC and News data 
        tmp_stocks_symbol = list(set([i[0] for i in tmp_stocks]))
        save_webscape_data(cnt, tmp_stocks_symbol)

        print('Finished {}-th iteration with {} seconds'.format(cnt, round(time.time()-start_time, 2)), '\n')

        cnt += 1
        print('Sleep {}mins more before the next iteration, and total {}mins...'.format(round((time_interval*60 - (time.time() - start_time))/60.0 ,2), time_interval))

        time.sleep(time_interval*60 - (time.time() - start_time)) # pull current stock prices with an interval 15mins

    
    except:
        print('Lost connection...')
        print('Sleep 60seconds and try again...')
        time.sleep(60)
        continue
        
    #time.sleep(sleep_period) # update every 60s
    


In [247]:
with open('{}/1_large_change_stocks.json'.format(tmp_path), 'w') as fp:
    json.dump(large_change_stocks_all, fp, indent = 4)
    
with open('{}/2_current_prices.json'.format(tmp_path), 'w') as fp:
    json.dump(current_prices, fp, indent = 4)
    
with open('{}/3_Readme.txt'.format(tmp_path), 'w') as fp:
    fp.write('Pulling stock prices every {} mins...'.format(time_interval))